In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [2]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [3]:
## test reading text files instead of converting to .csv using excel

In [4]:
path = Path('../SEC_Text/cnsfails201910a.txt')
df = pd.read_table(
    Path('../SEC_Text/cnsfails201910a.txt'),
    sep="|",
    #header=None,
    index_col=None
                )
df

,SETTLEMENT DATE,CUSIP,SYMBOL,QUANTITY (FAILS),DESCRIPTION,PRICE
0,20191001,B38564108,EURN,853.0,EURONAV NV ANTWERPEN (BELGIUM),9.20
1,20191001,D18190898,DB,142430.0,DEUTSCHE BANK AG NAMEN AKT (DE,7.50
2,20191001,G0080J112,ACTTU,307.0,ACT II GLOBAL ACQ CORP USD UIT,10.40
3,20191001,G01125106,AFYA,106.0,AFYA LTD CL A COM,25.64
4,20191001,G0177J108,AGN,819.0,ALLERGAN PLC ORD SHS (IRL),168.29
...,...,...,...,...,...,...
34124,20191011,98983V106,ZUO,3242.0,ZUORA INC CL A COM,14.52
34125,20191011,98985W102,ZYME,3160.0,ZYMEWORKS INC COM,26.80
34126,20191011,98986M103,ZYXI,125613.0,"ZYNEX, INC. COMMON STOCK (NV)",11.14
34127,Trailer record count 34127,NaN,NaN,NaN,NaN,NaN


In [5]:
df = df.iloc[:-2 , :] ## Drop last two rows as they are useless to keep
df = df.copy()
df.drop(['DESCRIPTION','PRICE'], axis=1,inplace=True)
df.rename(columns={'SETTLEMENT DATE':'Date','QUANTITY (FAILS)':'QUANTITY_FAILS'},inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,CUSIP,SYMBOL,QUANTITY_FAILS
0,2019-10-01,B38564108,EURN,853.0
1,2019-10-01,D18190898,DB,142430.0
2,2019-10-01,G0080J112,ACTTU,307.0
3,2019-10-01,G01125106,AFYA,106.0
4,2019-10-01,G0177J108,AGN,819.0
...,...,...,...,...
34122,2019-10-11,98980G102,ZS,11084.0
34123,2019-10-11,98980L101,ZM,349275.0
34124,2019-10-11,98983V106,ZUO,3242.0
34125,2019-10-11,98985W102,ZYME,3160.0


In [6]:
def read_ftd_file(file_path):
    df = pd.read_table(
    Path(file_path),
    sep="|",
    index_col=None
    )
    
    df = df.iloc[:-2 , :] ## Drop last two rows as they are useless to keep
    df.drop(['DESCRIPTION','PRICE'], axis=1,inplace=True)
    df.rename(columns={'SETTLEMENT DATE':'Date','QUANTITY (FAILS)':'QUANTITY_FAILS'},
             inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values(["SYMBOL", "Date"], ascending = (True, True),inplace=True)
    df.set_index('Date',inplace=True)
    
    return df

In [7]:
## Test function
test_df = read_ftd_file(path)
test_df

,CUSIP,SYMBOL,QUANTITY_FAILS
Date,,,
2019-10-07,00846U101,A,76.0
2019-10-08,00846U101,A,2294.0
2019-10-09,00846U101,A,9.0
2019-10-11,00846U101,A,20765.0
2019-10-10,013872106,AA,2830.0
...,...,...,...
2019-10-07,98986M103,ZYXI,78526.0
2019-10-08,98986M103,ZYXI,142473.0
2019-10-09,98986M103,ZYXI,105236.0


In [8]:
## Do Loop 
path = r'C:\Users\watso\OneDrive\Projects\FintechProject2_TWbranch\Fintech_Project_2\SEC_Text' # USE YOUR PATH 
all_files = glob.glob(path + "/*.txt")

ftd_df = pd.DataFrame()

for filename in all_files:
    if filename == 0:
        ftd_df = read_ftd_file(filename)
    else:
        ftd_df2 = read_ftd_file(filename)
        ftd_df = pd.concat([ftd_df,ftd_df2],axis='rows')

ftd_df

,CUSIP,SYMBOL,QUANTITY_FAILS
Date,,,
2019-10-07,00846U101,A,76.0
2019-10-08,00846U101,A,2294.0
2019-10-09,00846U101,A,9.0
2019-10-11,00846U101,A,20765.0
2019-10-10,013872106,AA,2830.0
...,...,...,...
2019-12-24,98986M103,ZYXI,73286.0
2019-12-26,98986M103,ZYXI,88268.0
2019-12-27,98986M103,ZYXI,104928.0


In [9]:
ftd_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 234195 entries, 2019-10-07 to 2019-12-31
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   CUSIP           234195 non-null  object 
 1   SYMBOL          234195 non-null  object 
 2   QUANTITY_FAILS  234195 non-null  float64
dtypes: float64(1), object(2)
memory usage: 7.1+ MB


In [10]:
cusip_symbol_df = ftd_df[['SYMBOL','CUSIP']]
cusip_symbol_df = cusip_symbol_df.drop_duplicates()
cusip_symbol_df = cusip_symbol_df.sort_values('SYMBOL')
cusip_symbol_df = cusip_symbol_df.reset_index(drop=True)
cusip_symbol_df

,SYMBOL,CUSIP
0,3118REORGPAY,G48833118
1,3126REORGPAY,G48833126
2,A,00846U101
3,AA,013872106
4,AAAU,715426102
...,...,...
12712,ZXAIY,16951E104
12713,ZYME,98985W102
12714,ZYNE,98986X109
12715,ZYXI,98986M103


In [11]:
# ## Export DFs
# ftd_df.to_csv('../Resources/ftd_all_data.csv')
# cusip_symbol_df.to_csv('../Resources/symbol_all_list.csv')
# save_obj(ftd_df, '../Resources/ftd_all_data.pkl')
# save_obj(cusip_symbol_df, '../Resources/symbol_all_list.pkl')